# Stochastic Simulation Assignment 2
- Dennis Curti (14526263)
- Sjoerd Dronkers (13420607)

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import simpy as sim
import scipy.stats as sp

## Question 2

In [75]:
waiting_times = list()

In [82]:
class Servers(object):
    """
    Class to simulate behaviour of the servers.
    """

    def __init__(self, env, n, service_time):
        """
        Initialize Server object.
        """

        self.env = env
        self.server = sim.Resource(env, n)
        self.service_time = service_time
        self.waiting_times = list()

    def processing(self, service_time):
        yield self.env.timeout(self.service_time)


In [84]:
def customer(env, servers, name, service_time):
    """
    Function to act as a customer.
    """

    arrival = env.now

    with servers.server.request() as request:
        yield request

        yield env.process(servers.processing(service_time))
        enter_time = env.now
        servers.waiting_times.append(enter_time - arrival - service_time)
        # global waiting_times
        # waiting_times.append(enter_time - arrival - service_time)
        # print(f'Waiting_time: {enter_time - arrival - service_time}')

In [93]:
def setup(env, servers, n, no_customers, service_time, arrival_rate):
    """
    Main function to run the system.
    """

    for i in range(no_customers):
        env.process(customer(env, servers, f'Person {i}', service_time))
        yield env.timeout(arrival_rate)

In [97]:
# Base parameters
# n = [1, 2, 4]
no_customers = 20
total_time = 500
n = 1
service_time = 10
arrival_rate = 5
run_time = 1000

In [98]:
def simulation(n, no_customers, service_time, arrival_rate, run_time):
    env = sim.Environment()
    servers = Servers(env, n, service_time)
    env.process(setup(env, servers, n, no_customers, service_time, arrival_rate))
    env.run(until=run_time)
    return servers.waiting_times

In [107]:
# Check if it works for simple cases
assert simulation(1, 2, 10, 5, 1000)[1] == 5
assert simulation(2, 2, 10, 5, 1000)[1] == 0
assert simulation(2, 3, 10, 3, 1000)[2] == 4

In [99]:
times = simulation(n, no_customers, service_time, arrival_rate, run_time)

In [100]:
times

[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95]

In [80]:
print(waiting_times)

[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95]


## Question 3

In [ ]:
# Useful for small first scheduling: sim.PriorityResource

## Question 4